## Initial Setup

In [223]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

input\top100.json


In [224]:
df = pd.read_json('input/top100.json')
df.head()

,id,rank,artist,song,rank_last_week,peak_rank,weeks_on_chart,date
0,1,1,Mariah Carey,All I Want For Christmas Is You,1 Last Week,1 Peak Rank,37 Weeks on Chart,2020-01-04
1,2,2,Brenda Lee,Rockin' Around The Christmas Tree,2 Last Week,2 Peak Rank,32 Weeks on Chart,2020-01-04
2,3,3,Bobby Helms,Jingle Bell Rock,9 Last Week,3 Peak Rank,30 Weeks on Chart,2020-01-04
3,4,4,Burl Ives,A Holly Jolly Christmas,6 Last Week,4 Peak Rank,15 Weeks on Chart,2020-01-04
4,5,5,Post Malone,Circles,3 Last Week,1 Peak Rank,17 Weeks on Chart,2020-01-04


In [225]:
len(df.loc[df['rank_last_week'] == df['weeks_on_chart']]) == len(df)


False

In [226]:
df['rank_last_week'] = df['rank_last_week'].apply(lambda rank_last_week: rank_last_week[:-10])
df['peak_rank'] = df['peak_rank'].apply(lambda peak_rank: peak_rank[:-10])
df['weeks_on_chart'] = df['weeks_on_chart'].apply(lambda weeks_on_chart: weeks_on_chart[:-15])

In [227]:
df['date'].dtype

dtype('<M8[ns]')

In [228]:
unique_artists = df['artist'].unique()
len(unique_artists)


451

In [229]:
unique_song_names = df['song'].unique()
len(unique_song_names)


742

In [230]:
df.dtypes

id                         int64
rank                       int64
artist                    object
song                      object
rank_last_week            object
peak_rank                 object
weeks_on_chart            object
date              datetime64[ns]
dtype: object

In [231]:
df.columns

Index(['id', 'rank', 'artist', 'song', 'rank_last_week', 'peak_rank',
       'weeks_on_chart', 'date'],
      dtype='object')

In [232]:
artist_song_pt = pd.pivot_table(df, index=['artist', 'song'], values=['peak_rank', 'weeks_on_chart'], aggfunc={'peak_rank' : 'min', 'weeks_on_chart' : 'max'})
artist_song_pt

peak_rank  \
artist                     song                                       
21 Savage & Metro Boomin   Brand New Draco                       57   
                           Glock In My Lap                       19   
                           Many Men                              33   
                           My Dawg                               56   
                           No Opp Left Behind                    71   
...                                                             ...   
Zac Brown Band             The Man Who Loves You The Most        96   
Zayn                       Better                                89   
blackbear                  Hot Girl Bummer                       11   
surf mesa Featuring Emilee ily                                  100   
twenty one pilots          Level Of Concern                      23   

                                                          weeks_on_chart  
artist                     song                                           
21 Savage & Metro Boomin   Brand New Draco                             1  
                           Glock In My Lap                             3  
                           Many Men                                    2  
                           My Dawg                                     1  
                           No Opp Left Behind                          1  
...                                                                  ...  
Zac Brown Band             The Man Who Loves You The Most              1  
Zayn                       Better                                      1  
blackbear                  Hot Girl Bummer                            42  
surf mesa Featuring Emilee ily                                         9  
twenty one pilots          Level Of Concern                            9  

[751 rows x 2 columns]

In [257]:
artist_song_list = []

for row in artist_song_pt.itertuples():
    artist_song_list.append(row)

artist_song = pd.DataFrame(artist_song_list)

artist_song[['artist', 'song']] = pd.DataFrame(artist_song['Index'].tolist())

artist_song = artist_song[['artist', 'song', 'weeks_on_chart', 'peak_rank']]
artist_song.describe()


,artist,song,weeks_on_chart,peak_rank
count,751,751,751,751
unique,451,742,40,100
top,Lil Uzi Vert,Circles,1,1
freq,24,3,298,31


In [298]:
censored_songnames = artist_song.loc[artist_song['song'].str.contains('\*')]['song']
censored_songnames = censored_songnames.tolist()

censored_words = []
for songname in censored_songnames:
        words = songname.split(' ')
        for word in words:
            if '*' in word:
                censored_words.append(word)

print(censored_words)
print(len(censored_words))

['N*ggas', 'N*gga', 'Sh*t', 'SH*T', 'P*$$y', 'N**gas', 'B*tch', 'F*ck']
8
Bla bla bla Niggas yeh yeh


In [ ]:
#manual solution due to small size of list, may change later
censorship = {
                 'N*ggas' : 'Niggas',
                 'N*gga' : 'Nigga',
                 'Sh*t' : 'Shit',
                 'SH*T' : 'SHIT',
                 'P*$$y' : 'Pu$$y',
                 'N**gas' : 'Niggas',
                 'B*tch' : 'Bitch',
                 'F*ck' : 'Fuck'
}


def get_uncensored_string(censored_string):
    uncensored_words = []
    words = censored_string.split(' ')
    for word in words:
        if word in censorship:
            word = censorship[word]
        uncensored_words.append(word)
    return ' '.join(uncensored_words)


In [303]:
artist_song.iloc[9]

artist            21 Savage & Metro Boomin
song                     Steppin On N*ggas
weeks_on_chart                           1
peak_rank                               74
Name: 9, dtype: object

In [304]:
artist_song['song'] = artist_song['song'].apply(lambda song: get_uncensored_string(song))
artist_song.iloc[9]

artist            21 Savage & Metro Boomin
song                     Steppin On Niggas
weeks_on_chart                           1
peak_rank                               74
Name: 9, dtype: object

In [306]:
import lyricsgenius
token_file = open("genius_token.txt", "r")
token = token_file.read()
token_file.close()

genius = lyricsgenius.Genius(token)
genius.verbose = False
genius.remove_section_headers = True

#this is going to be very time consuming
#artist_song['lyrics'] = np.vectorize(genius.search_song)(artist_song['song'],artist_song['artist'])

#testing lyricsgenius
def get_lyrics(title, artist):
    song = genius.search_song(title, artist)
    lyrics = song.lyrics.replace('\n', ' ').replace('\r', '')
    return lyrics

test_df = artist_song.copy().iloc[0:10]
test_df['lyrics'] = np.vectorize(get_lyrics)(test_df['song'],test_df['artist'])
test_df

,artist,song,weeks_on_chart,peak_rank,lyrics
0,21 Savage & Metro Boomin,Brand New Draco,1,57,"Yeah ( Metro ) We ain't giving out smoke, we g..."
1,21 Savage & Metro Boomin,Glock In My Lap,3,19,"Y'all niggas stop playin', nigga Y'all niggas ..."
2,21 Savage & Metro Boomin,Many Men,2,33,"Metro Ayy, ayy, ayy, ayy, ayy, ayy I kept it ..."
3,21 Savage & Metro Boomin,My Dawg,1,56,"Just because I didn't remind you, don't think ..."
4,21 Savage & Metro Boomin,No Opp Left Behind,1,71,Metro in this bitch goin' brazy Made it out t...
5,21 Savage & Metro Boomin,RIP Luv,1,76,"Yeah, rest in peace to love October 22nd, 1992..."
6,21 Savage & Metro Boomin,Runnin,7,9,Am I really hard to please? Perhaps I have suc...
7,21 Savage & Metro Boomin,Said N Done,1,91,[Intro: Young Thug ] ( Metro Boomin want some...
8,21 Savage & Metro Boomin,Slidin,2,32,"Uh-huh, uh-huh, uh-huh, uh-huh Uh-huh, uh-huh,..."
9,21 Savage & Metro Boomin,Steppin On Niggas,1,74,Step on a nigga in my Air Force 1s Step on a n...
